In [14]:
from pymoo.problems.many.dtlz import DTLZ2
from pymoo.util.ref_dirs import get_reference_directions
import pandas as pd

In [5]:
problem = DTLZ2(n_obj=2)
ref_dirs = get_reference_directions("das-dennis", 2, n_partitions=50)

In [11]:
output = {}
problem._evaluate(ref_dirs,output)

In [17]:
objective_value_1 = [i[0] for i in output['F']]
objective_value_2 = [i[1] for i in output['F']]
x1 = [i[0] for i in ref_dirs]
x2 = [i[1] for i in ref_dirs]
dict_data = {}

dict_data['x1'] = x1
dict_data['x2'] = x2
dict_data['Objective 1']  = objective_value_1
dict_data['Objective 2'] = objective_value_2
dataset = pd.DataFrame(data=dict_data)

In [19]:
dataset.head()

,x1,x2,Objective 1,Objective 2
0,0.00,1.00,1.250000,0.000000
1,0.02,0.98,1.229793,0.038648
2,0.04,0.96,1.209209,0.076077
3,0.06,0.94,1.188303,0.112328
4,0.08,0.92,1.167124,0.147442
